#  문장 내의 단어들을 임베딩
- Huggingface 무료 모델 사용
https://huggingface.co/nlpai-lab/KURE-v1

In [1]:
from sentence_transformers import SentenceTransformer

# Hugging Face Hub에서 한국어 임베딩 모델 다운로드
# KURE-v1은 한국어에 특화된 문장 임베딩 모델입니다
model = SentenceTransformer("nlpai-lab/KURE-v1")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [2]:
# 샘플 데이터: 간단한 문장들의 모음
sentences = [
    '나는 인공지능 공부를 좋아한다.',
    '인공지능은 매우 흥미롭다.',
    '오늘 날씨가 흐리고 비가 온다.'
]

# 문장들을 벡터로 변환 (임베딩 생성)
# encode() 메서드는 각 문장을 고정 길이의 숫자 벡터로 변환합니다
embeddings = model.encode(sentences)

# 임베딩의 형태(shape) 출력
# 결과: (문장 개수, 임베딩 차원) 형태로 출력됨
print(embeddings.shape)

(3, 1024)


In [3]:
# 임베딩 결과 확인
print('첫 번째 문장의 임베딩 벡터 (처음 10개 값):')
print(embeddings[0][:10])

첫 번째 문장의 임베딩 벡터 (처음 10개 값):
[-0.04133056 -0.0088662  -0.01965634  0.00259992 -0.01420301 -0.00173231
  0.00808719 -0.02382692  0.03474926  0.01797114]


In [4]:
import pandas as pd

# 임베딩 결과를 DataFrame으로 변환하여 시각화
embedding_df = pd.DataFrame(embeddings, index=sentences)
print(embedding_df.head())

                       0         1         2         3         4         5     \
나는 인공지능 공부를 좋아한다. -0.041331 -0.008866 -0.019656  0.002600 -0.014203 -0.001732   
인공지능은 매우 흥미롭다.    -0.031890  0.007180 -0.015229  0.003641 -0.032137 -0.021120   
오늘 날씨가 흐리고 비가 온다. -0.027240  0.072480 -0.033453 -0.013671 -0.020261 -0.046810   

                       6         7         8         9     ...      1014  \
나는 인공지능 공부를 좋아한다.  0.008087 -0.023827  0.034749  0.017971  ...  0.009031   
인공지능은 매우 흥미롭다.     0.033232 -0.034247  0.014975  0.013564  ...  0.015914   
오늘 날씨가 흐리고 비가 온다. -0.030745  0.014778 -0.006895 -0.017058  ... -0.018110   

                       1015      1016      1017      1018      1019      1020  \
나는 인공지능 공부를 좋아한다. -0.015729  0.076318  0.038932 -0.020271 -0.014609  0.006914   
인공지능은 매우 흥미롭다.    -0.002616  0.036231  0.045487 -0.049278 -0.017388 -0.012169   
오늘 날씨가 흐리고 비가 온다.  0.025783  0.003339  0.059365 -0.023501 -0.008735 -0.011778   

                       1021      1022      10

In [5]:
# 임베딩 간의 유사도 점수를 계산합니다
# similarity_matrix는 각 임베딩 벡터 간의 코사인 유사도를 담은 행렬입니다
similarity_matrix = model.similarity(embeddings, embeddings)
print(similarity_matrix)

tensor([[1.0000, 0.7380, 0.4107],
        [0.7380, 1.0000, 0.4417],
        [0.4107, 0.4417, 1.0000]])


In [6]:
# 문장 간 코사인 유사도를 출력합니다
print('\n문장 간 코사인 유사도:')

# 모든 문장 쌍에 대해 유사도를 계산하고 출력
# 중복을 피하기 위해 상삼각 행렬만 순회 (i < j인 경우만)
for i in range(len(sentences)):
    for j in range(i+1, len(sentences)):
        # i번째 문장과 j번째 문장 간의 유사도 값 추출
        similarity = similarity_matrix[i][j]
        # 두 문장과 그들 간의 유사도를 소수점 4자리까지 출력
        print(f'{sentences[i]} vs {sentences[j]}: {similarity:.4f}')


문장 간 코사인 유사도:
나는 인공지능 공부를 좋아한다. vs 인공지능은 매우 흥미롭다.: 0.7380
나는 인공지능 공부를 좋아한다. vs 오늘 날씨가 흐리고 비가 온다.: 0.4107
인공지능은 매우 흥미롭다. vs 오늘 날씨가 흐리고 비가 온다.: 0.4417
